In [1]:
!pip install langchain

In [24]:
## Open AI API key and Open Source models Llama3, gemma2, mistral-- Groq
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [7]:
!pip install langchain_groq

In [ ]:
## Langchain wants to create integration with all llm models
##from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
## 9B paramater version of Google Gemma models
model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

 

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a913760>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a9112d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
## LLM model is like if you give a query, you need an answer in return
!pip install langchain_core

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage
## SystemMessage is an instruction to an LLM, and other is a message form human being

messages=[
    ##Instructing our LLM model to do this
    SystemMessage(content="Translate the content from English to French"),
    ## What human is speaking
    HumanMessage(content="Hello! How are you?")
]

## now invoke
result=model.invoke(messages)

In [16]:
result

AIMessage(content='Bonjour ! Comment vas-tu ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 22, 'total_tokens': 33, 'completion_time': 0.02, 'prompt_time': 0.00216314, 'queue_time': 0.258201361, 'total_time': 0.02216314}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7e619122-ac98-4b48-9345-1afbb023dc65-0', usage_metadata={'input_tokens': 22, 'output_tokens': 11, 'total_tokens': 33})

In [17]:
## Its giving long responses like including meta datas as well, we just need translation
from langchain_core.output_parsers import StrOutputParser
##it will just display the content message out
parser=StrOutputParser()
parser.invoke(result)

'Bonjour ! Comment vas-tu ? \n'

In [18]:
## We have 2 comp- 1) Model 2)StrOutputParser
## So use LCEL and chain the components
chain=model|parser
chain.invoke(messages)

"Hello! How are you?\n\n**French:**\n\nBonjour ! Comment vas-tu ? \n\n\nLet me know if you have any other phrases you'd like me to translate!\n"

In [19]:
## Prompt templates, making a reusable prompt template
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    ## Here also System and User
    [("system", generic_template),("user", "{text}")]
)

In [21]:
result=prompt.invoke({"language":"French", "text":"Hello"})

In [22]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [ ]:
##Chain together iwth LCEL
chain=prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

"Bonjour 👋 \n\n\nLet me know if you have any other words or phrases you'd like translated! 😊\n"

In [ ]:
## To convert all these things to APIs will use, LangServe
## RESTAPI, lets backedn and frontend talk to each other 